# Arbeitspaket (AP) 3: Management & Nutzung Räumliche Daten

### Angaben Studierende(r) (fehlende Angaben ergänzen)

<table>
  <tr>
    <td>Vorname:</td>
    <td></td>
  </tr>
  <tr>
    <td>Nachname:</td>
    <td></td>
  </tr>
  <tr>
    <td>Immatrikulationsnummer:</td>
    <td></td>
  </tr>
  <tr>
    <td>Modul:</td>
    <td>Data Science</td>
  </tr>
  <tr>
    <td>Prüfungsdatum / Raum / Zeit:</td>
    <td>06.10.2025 / Raum: MU O2.001 / 8:00 – 11:45</td>
  </tr>
  <tr>
    <td>Erlaubte Hilfsmittel:</td>
    <td>w.MA.XX.DS.25HS (Data Science)<br>Open Book, Eigener Computer, Internet-Zugang</td>
  </tr>
  <tr>
  <td>Nicht erlaubt:</td>
  <td>Nicht erlaubt ist der Einsatz beliebiger Formen von generativer KI (z.B. Copilot, ChatGPT) <br> sowie beliebige Formen von Kommunikation oder Kollaboration mit anderen Menschen.</td>
</tr>
</table>

## Bewertungskriterien

### <b style="color: gray;">(max. erreichbare Punkte: 40)</b>

<table>
  <thead>
    <tr>
      <th>Kategorie</th>
      <th>Beschreibung</th>
      <th>Punkteverteilung</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Code nicht lauffähig oder Ergebnisse nicht sinnvoll</td>
      <td>Der Code enthält Fehler, die verhindern, dass er ausgeführt werden kann (z.B. Syntaxfehler) oder es werden Ergebnisse ausgegeben, welche nicht zur Fragestellung passen.</td>
      <td>0 Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit gravierenden Mängeln</td>
      <td>Der Code läuft, aber die Ergebnisse sind aufgrund wesentlicher Fehler unvollständig (z.B. fehlende Joins, gravierende Fehler in SQL-Abfragen). Nur geringer Fortschritt erkennbar.</td>
      <td>25% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit mittleren Mängeln</td>
      <td>Der Code läuft und liefert teilweise korrekte Ergebnisse, aber es gibt grössere Fehler (z.B. fehlende Spalten, unvollständige SQL-Abfragen). Die Ergebnisse sind nachvollziehbar, aber unvollständig oder ungenau.</td>
      <td>50% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit minimalen Mängeln</td>
      <td>Der Code läuft und liefert ein weitgehend korrektes Ergebnis, aber kleinere Fehler (z.B. falsche oder fehlende Sortierung, Rundung von Werten falsch) beeinträchtigen die Vollständigkeit des Ergebnisses.</td>
      <td>75% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig und korrekt</td>
      <td>Der Code läuft einwandfrei und liefert das korrekte Ergebnis ohne Mängel.</td>
      <td>100% der max. erreichbaren Punkte</td>
    </tr>
  </tbody>
</table>


#### <b>Python Libraries und Settings</b>

In [1]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from IPython.display import display, IFrame
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())


/workspaces/python_postgresql_postgis/Leistungsnachweis 1


## <b>Vorbereitung (Hinweis: dieser Teil wird <u>nicht</u> bewertet)</b>

#### <b>1.) Starten Sie eine GitHub Codespaces Instanz auf Basis Ihres Forks des folgenden GitHub Repositories:</b>

##### GitHub-Repository: https://github.com/mario-gellrich-zhaw/python_postgresql_postgis

##### <span style="color: red;"><b>WICHTIG!!! Verwenden Sie eine GitHub Codespaces Instanz mit ausreichend Arbeitsspeicher (4core, 16GB RAM).</b></span>

<b>Hinweis:</b> 
- Im Unterricht wurden bereits sämtliche Installationen und Einstellungen inkl. der Registrierung des Datenbank Servers auf pgAdmin vorgenommen.
- Falls Sie die Codespaces-Instanz neu erstellen müssen, folgen Sie bitte den detaillierten Erklärungen auf der README-Seite des GitHub Repositories.

#### <b>2.) Erstellen und Testen Sie die Datenbankverbindung mit der 'osm_switzerland' Datenbank.</b>

In [2]:
# Set up Database Connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Erstellen der Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Erstellen SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test der Connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT current_database();"))
        db_name = result.scalar()
        print("Aktuelle Datenbank:", db_name)
except Exception as e:
    print("Fehler bei der Verbindung:", e)

# Verbindung trennen
engine.dispose()

Aktuelle Datenbank: osm_switzerland


#### <b>3.) Legen Sie einen neuen Order 'maps' im Arbeitsverzeichnis an. In diesem werden Ihre generierten Karten abgespeichert.</b>

<b style="color:red;">Hinweis: Der Order 'maps' inklusive die erzeugten Karten muss als Teil des AP03 mit auf EXAM-Moodle abgegeben werden!</b>

In [3]:
# Anlegen eines Ordners 'maps'
if not os.path.exists('maps'):
    os.makedirs('maps')
    print("Ordner 'maps' wurde erstellt.")
else:
    print("Ordner 'maps' existiert bereits und wird nicht neu angelegt!")

Ordner 'maps' wurde erstellt.


## <b>Aufgaben (Dieser Teil wird bewertet!)</b>
<b>Hinweise zu den folgenden Aufgabenstellungen:</b>
<ul>
  <li>In diesem Jupyter Notebook gibt es jeweils zwei Code-Zellen pro Aufgabe:</li>
  <ol>
    <li>Eine Codezelle mit Python-Code für die SQL-Abfrage.</li>
    <li>Eine Codezelle mit Python-Code für die Kartendarstellung der Ergebnisse der jeweiligen SQL-Abfrage.</li>
  </ol>
  <li>In den Codezellen für die SQL-Abfrage muss jeweils die SQL-Statement ergänzt werden.</li>
  <li>In den Codezellen für die Kartendarstellung muss nur dann der Python Code ergänzt werden, wenn in der Aufgabe danach gefragt wird.</li>
</ul>
<b style="color:red;">Beachten Sie, dass für die Punktevergabe auch die weiteren Anforderungen zu den Fragen unter 'Details zur Aufgabenstellung' herangezogen werden.</b>

#### <b>Aufgabe (1): Erstellen Sie eine Abfrage der Spitäler.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Daten in der Tabelle 'planet_osm_polygon'.
- Stellen sie in der Ergebnistabelle die Spalten: osm_id, amenity sowie die transformierte Geometrie als Spalte geom dar.
- Tipps:
    - Die Geometry wird mit Hilfe der Funktion st_transform() transformiert, z.B.: *st_transform(p.way, 4326) AS geom*.
    - Die Daten sind mit dem key:value Paar amenity='hospital' in der OpenStreetMap Map-Feature Übersicht angegeben.
    - vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [5]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.amenity,
            st_transform(p.way, 4236) AS geom
         FROM
            planet_osm_polygon AS p
         WHERE p.amenity = 'hospital';"""

# Ergebnis in GeoDataFrame abspeichern
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf

,osm_id,amenity,geom
0,856698475,hospital,"POLYGON ((8.57123 47.4466, 8.5717 47.44648, 8...."
1,-18241258,hospital,"POLYGON ((8.57801 47.35114, 8.57821 47.35019, ..."
2,125667230,hospital,"POLYGON ((8.58185 47.34925, 8.5819 47.34914, 8..."
3,675191947,hospital,"POLYGON ((8.6024 47.34505, 8.60313 47.34504, 8..."
4,673595876,hospital,"POLYGON ((8.73285 47.34943, 8.73288 47.3494, 8..."
5,-17941802,hospital,"POLYGON ((8.72391 47.2669, 8.72427 47.26661, 8..."
6,-9065183,hospital,"POLYGON ((8.46295 47.28081, 8.46324 47.28054, ..."
7,-9065183,hospital,"POLYGON ((8.46101 47.28284, 8.4611 47.2826, 8...."
8,609537142,hospital,"POLYGON ((8.52415 47.39577, 8.52417 47.39573, ..."
9,42156983,hospital,"POLYGON ((8.52315 47.41216, 8.52344 47.41209, ..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [8]:
# Projektion definieren (WGS84)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='CartoDBPositron')

# Map settings
folium.GeoJson(
    gdf,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_hospital.html')
print("Die Karte wurde unter 'maps/map_hospital.html' gespeichert.")

Die Karte wurde unter 'maps/map_hospital.html' gespeichert.


#### <b>Aufgabe (2) Erstellen Sie eine Abfrage der SBB Bahnhöfe.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_point'.
- Schränken Sie nach Bahnhöfen der SBB ein.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, name, operator, public_transport und die transformierte Geometrie als Spalte geom dar.
- Tipp: Die Daten sind mit dem key:value Paar public_transport='station' in der OpenStreetMap Map-Feature Übersicht angegeben.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [22]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.name,
            p.operator,
            p.public_transport,
            st_transform(p.way, 4236) AS geom
         FROM
            planet_osm_point AS p
         WHERE p.public_transport = 'station'
         AND p.operator = 'SBB';"""

# Ergebnis in GeoDataFrame abspeichern
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf


,osm_id,name,operator,public_transport,geom
0,4574932116,Rüti,SBB,station,POINT (8.86023 47.25769)
1,2414238691,Tann-Dürnten,SBB,station,POINT (8.85832 47.2658)
2,2443333883,Steg,SBB,station,POINT (8.93825 47.35121)
3,2141115746,Fischenthal,SBB,station,POINT (8.92916 47.33075)
4,762194590,Bauma,SBB,station,POINT (8.88409 47.36615)
...,...,...,...,...,...
124,2427867574,Schloss Laufen am Rheinfall,SBB,station,POINT (8.61999 47.6738)
125,1317463581,Hettlingen,SBB,station,POINT (8.70069 47.54268)
126,1313973484,Seuzach,SBB,station,POINT (8.74494 47.53296)
127,2455946411,Ossingen,SBB,station,POINT (8.73003 47.61191)


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [23]:
# Projektion definieren (WGS84)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=10, 
               tiles='CartoDBPositron')

# Map settings with popup
folium.GeoJson(
    gdf,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_stations.html')
print("Die Karte wurde unter 'maps/map_stations.html' gespeichert.")

Die Karte wurde unter 'maps/map_stations.html' gespeichert.


#### <b>Aufgabe (3): Erstellen Sie eine Abfrage aller Strassen, die als 'primary', 'secondary' oder 'tertiary' klassifiziert sind mit Infos zur Art des Fahrradwegs.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'planet_osm_line'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, highway, bicycle und die transformierte Geometrie als Spalte geom dar.
- Schränken Sie die Abfrage auf alle Strassen ein, bei denen Information zur Art des Fahrradweges vorliegen.
- Tipp: Sie finden die Informationen zur Art der Fahrradwegs in der Spalte 'bicycle'.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [ ]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.highway,
            p.bicycle,
            st_transform(p.way, 4326) AS geom
         FROM
            planet_osm_line AS p
      WHERE highway = 'primary'
         ORDER BY highway DESC
     ;"""

# Ergebnis in GeoDataFrame abspeichern
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf

,osm_id,highway,bicycle,geom
0,315007668,primary,use_sidepath,"LINESTRING (8.83126 47.24526, 8.83146 47.24537..."
1,1188796458,primary,use_sidepath,"LINESTRING (8.83238 47.24567, 8.83257 47.24569..."
2,1188796459,primary,use_sidepath,"LINESTRING (8.83489 47.24705, 8.83486 47.24703)"
3,315007660,primary,use_sidepath,"LINESTRING (8.83486 47.24693, 8.83496 47.247)"
4,31726656,primary,use_sidepath,"LINESTRING (8.83496 47.247, 8.83522 47.24717, ..."
...,...,...,...,...
3307,700677161,primary,None,"LINESTRING (8.82272 47.53991, 8.82259 47.53987..."
3308,955364124,primary,None,"LINESTRING (8.82403 47.54032, 8.82392 47.54029..."
3309,955364112,primary,use_sidepath,"LINESTRING (8.81843 47.53988, 8.81815 47.53983)"
3310,216616020,primary,None,"LINESTRING (8.82234 47.53981, 8.82205 47.53976..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [41]:
# Projektion definieren (WGS84)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='CartoDBPositron')

# Map settings
folium.Choropleth(
    geo_data=gdf,
    name='map',
    line_weight=3,
    line_color='darkred'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_streets.html')
print("Die Karte wurde unter 'maps/map_streets.html' gespeichert.")

Die Karte wurde unter 'maps/map_streets.html' gespeichert.


#### <b>Aufgabe (4): Erstellen Sie eine Abfrage aller Gebäude in der Bahnhofstrasse in Zürich, bei denen mindestens die Postleitzahl (PLZ) vorhanden ist.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_polygon'.
- Stellen Sie in der Ergebnistabelle Strassenname, Haunummer, PLZ, Gemeindename sowie die transformierte Geometrie als Spalte geom dar.
- Dabei ist nicht wichtig, dass alle Adress-Attribute für jedes Gebäude vorhanden sind, nur die PLZ muss immer vorhanden sein.
- Verwenden Sie für die Darstellung als Hintergrundkarte ein Satellitenbild (ESRIWorldImagery) als maptile.
- Tipp: Gebäude sind in der Spalte 'building' klassifiziert. Mit *WHERE building IS NOT NULL* können Sie Gebäude filtern.

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [46]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p."addr:street",
            p."addr:housenumber",
            p."addr:city",
            p."addr:postcode",
            st_transform(p.way, 4326) AS geom
         FROM
            planet_osm_polygon AS p
            WHERE "addr:postcode" IS NOT NULL
            AND building IS NOT NULL
    
     ;"""

# Ergebnis in GeoDataFrame abspeichern
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf

,osm_id,addr:street,addr:housenumber,addr:city,addr:postcode,geom
0,261912959,Engelhölzlistrasse,15,Jona,8645,"POLYGON ((8.83055 47.24701, 8.83058 47.24699, ..."
1,261912956,Engelhölzlistrasse,27,Jona,8645,"POLYGON ((8.83075 47.24773, 8.83088 47.24767, ..."
2,204903629,Hüllistein,8,Rüti ZH,8630,"POLYGON ((8.8382 47.24717, 8.83833 47.24716, 8..."
3,204903645,Hüllistein,6,Rüti ZH,8630,"POLYGON ((8.83889 47.24757, 8.83898 47.24744, ..."
4,204903636,Hüllistein,3,Rüti ZH,8630,"POLYGON ((8.83876 47.24809, 8.83885 47.24808, ..."
...,...,...,...,...,...,...
166692,256572178,Liebensbergerstrasse,6,Gundetswil,8543,"POLYGON ((8.82448 47.5399, 8.82459 47.53983, 8..."
166693,256572202,Liebensbergerstrasse,1,Gundetswil,8543,"POLYGON ((8.82491 47.54015, 8.82508 47.54006, ..."
166694,256572205,Liebensbergerstrasse,3,Gundetswil,8543,"POLYGON ((8.82559 47.54002, 8.82567 47.53993, ..."
166695,255020279,Hofacker,1,Gundetswil,8543,"POLYGON ((8.82192 47.53893, 8.82193 47.53879, ..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [49]:
# Projektion definieren (WGS84)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='CartoDB positron')

# Map settings
folium.Choropleth(
    geo_data=gdf,
    name='map',
    fill_color='orange'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_buildings_Carto.html')
print("Die Karte wurde unter 'maps/map_buildings.html' gespeichert.")

Die Karte wurde unter 'maps/map_buildings.html' gespeichert.


#### <b>Aufgabe (5): Erstellen Sie eine Abfrage des Greifensees, und generieren Sie einen Buffer um diesen See mit einer Breite von 500m.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'planet_osm_polygon'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, name, natural sowie die transformierte Geometrie des Buffers als Spalte geom dar.
- Im Polygondatensatz gibt es unter dem Namen 'Greifensee' nicht nur einen Polygon. Filtern Sie in der Abfrage nur den See.
- Tipp: Zum Filtern des Sees können Sie die Spalte 'natural' verwenden.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [55]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.name,
            p.natural,
             ST_Transform(ST_Buffer(ST_Transform(p.way::geometry, 3857), 500), 4326) AS geom
         FROM
            planet_osm_polygon AS p
         WHERE
            p.name = 'Greifensee'
         AND p.natural = 'water'
     ;"""

# Ergebnis in GeoDataFrame abspeichern
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf

,osm_id,name,natural,geom
0,-1678346,Greifensee,water,"POLYGON ((8.6478 47.36811, 8.64782 47.3682, 8...."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [56]:
# Projektion definieren (WGS84)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='ESRIWorldImagery')

# Map settings
folium.Choropleth(
    geo_data=gdf,
    name='map',
    fill_color='greenyellow',
    fill_opacity=0.5,
    line_opacity=0.5
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_lake.html')
print("Die Karte wurde unter 'maps/map_lake.html' gespeichert.")

Die Karte wurde unter 'maps/map_lake.html' gespeichert.


#### <b>Aufgabe (6): Erstellen Sie eine Abfrage der landwirtschaftlichen Nutzflächen in den Grenzen der Stadt Zürich.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Daten in der Tabelle 'planet_osm_polygon'.
- Landwirtschaftliche Nutzflächen sollen in dieser Aufgabe sowohl 'farmland' als auch 'meadows' beinhalten.
- Selektieren Sie die landwirtschaftlichen Nutzflächen über die Gemeindefläche.
- Berechen Sie die Fläche der landwirtschaftlichen Nutzflächen und speichern diese in einer Spalte 'area_ha' ab.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, landuse, area_ha sowie die transformierte Geometrie als geom dar.
- Wählen Sie eine Satelliten Karte von ESRI als Hintergrundkarte (maptile).
- Ordnen Sie die Ergebnistabelle nach area_ha absteigend.
- Tipp: Im Unterricht wurde eine ähnliche Abfrage mit Waldflächen (forest) behandelt.
- vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [107]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.landuse,
            p.area,
             ST_Transform(p.way::geometry, 4326) AS geom
         FROM
            planet_osm_polygon AS p
            WHERE landuse = 'farmland'
         ORDER BY area DESC
     ;"""

# Ergebnis in GeoDataFrame abspeichern
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf

,osm_id,landuse,area,geom
0,977516381,farmland,None,"POLYGON ((8.83828 47.24343, 8.83872 47.24132, ..."
1,351872934,farmland,None,"POLYGON ((8.84429 47.24994, 8.84511 47.24987, ..."
2,351872933,farmland,None,"POLYGON ((8.84516 47.25038, 8.84595 47.25022, ..."
3,362601077,farmland,None,"POLYGON ((8.83948 47.24765, 8.83981 47.24704, ..."
4,361283959,farmland,None,"POLYGON ((8.85671 47.24575, 8.85693 47.24539, ..."
...,...,...,...,...
5313,285670607,farmland,yes,"POLYGON ((8.60613 47.65129, 8.60616 47.65116, ..."
5314,285670606,farmland,yes,"POLYGON ((8.60685 47.64917, 8.60806 47.64766, ..."
5315,285670604,farmland,yes,"POLYGON ((8.60806 47.64766, 8.60813 47.64757, ..."
5316,284697877,farmland,yes,"POLYGON ((8.59905 47.64185, 8.59907 47.64154, ..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [79]:
# Projektion definieren (WGS84)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_farmland_meadow.html')
print("Die Karte wurde unter 'maps/map_farmland_meadow.html' gespeichert.")

Die Karte wurde unter 'maps/map_farmland_meadow.html' gespeichert.


#### <b>Aufgabe (7): Erstellen Sie eine Abfrage der Hotels in einem Radius von 5km um den Flughafen Zürich.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_point'.
- Berechnen Sie in der Abfrage die Distanz jedes Hotels zum Flughafen in Metern als Spalte 'distance_meters'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, tourism, name, distance_meters, sowie die transformierte Geometrie als geom dar.
- Wählen Sie eine Satelliten Karte von ESRI als Hintergrundkarte (maptile).
- Integrieren Sie in die Kartendarstellung den Namen (Spalte 'name') der Hotels als Popup.
- Tipp: Hotels sind mit dem key:value Paar tourism='hotel' in der OpenStreetMap Map-Feature Übersicht angegeben.
- vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [94]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)  

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.tourism,
            p.name,
            p.amenity,
            ST_Distance(
                ST_Transform(p.way, 4326)::geography,
                ST_SetSRID(ST_MakePoint(8.560238, 47.452433), 4326)::geography
            ) AS distance_meters,
            ST_Transform(p.way, 4326) AS geom        
         FROM
            planet_osm_point AS p
         WHERE
            p.tourism = 'hotel'
            AND ST_DWithin(
                ST_Transform(p.way, 4326)::geography,
                ST_SetSRID(ST_MakePoint(8.560238, 47.452433), 4326)::geography,
                5000
        )
        ORDER BY distance_meters;"""


#mit ST_Distance(...) AS distance_meters berechent die Entfernung in Metern zwischen Suche und festem Punkt
#mit ST_Transform wandelt die Geometrie in Koordinatensystem, sodass GeoFrame es nutzen kann
#mit ST_MakePoint baut einen Punkt (hier Referenzpunkt HB) in den Code ein --> Reihenfolge immer: Longtitude (x), Latitude (y)
#mit ST_SetSRID(geometry,srid) weist einer Geometrie einen SRID zu (Spatial Reference System Identifier) --> ohne SRID weiss PostGIS nicht, wie Entfernungen und Transformationen gerechnet werden sollen.
#mit ST_DWithin(...,500) werden nur die Salons die maximal 500 meter vom Referenzpunkt entfernt sind. (Radius)

# Ergebnis in GeoDataFrame abspeichern
gdf = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf


,osm_id,tourism,name,amenity,distance_meters,geom
0,9553976171,hotel,Capsule Hotel Alpine Garden,None,190.680878,POINT (8.5627 47.45204)
1,8607233055,hotel,Transit Hotel & Dayrooms,None,208.034144,POINT (8.56018 47.45056)
2,291687394,hotel,Radisson Hotel Zurich Airport,None,1523.983696,POINT (8.54014 47.45099)
3,3086649239,hotel,Dorint Airport-Hotel Zürich,None,1577.322993,POINT (8.56559 47.43872)
4,3783549518,hotel,STAY@Zurich Airport,None,1588.795563,POINT (8.56747 47.43901)
5,6286669885,hotel,Welcome Inn,None,1651.984475,POINT (8.5816 47.44914)
6,12072254954,hotel,Sleepz,None,1731.824429,POINT (8.58278 47.44947)
7,1060256103,hotel,Holiday Inn Express Zürich Airport,None,1741.916573,POINT (8.53715 47.45186)
8,6286669886,hotel,Fly away,None,1830.280682,POINT (8.58426 47.45007)
9,4269704591,hotel,Apart-Hotel Zürich by Hilton,None,1871.576776,POINT (8.57373 47.43831)


In [95]:
# Projektion definieren (WGS84)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_hotels.html')
print("Die Karte wurde unter 'maps/map_hotels.html' gespeichert.")

Die Karte wurde unter 'maps/map_hotels.html' gespeichert.


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [109]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2025-10-06 09:41:15
Python Version: 3.12.1
IP Address: 127.0.0.1
-----------------------------------
